<a href="https://colab.research.google.com/github/escvelo/monorepo/blob/master/Interviews/PoseEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Change this path where data is located
DATA_DIR = "D:/interview/task_RespoVision/pose3d_data"

import cv2
import os
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

# Constants
SPIN_TO_COCO = [0, 16, 15, 18, 17, 5, 2, 6, 3, 7, 4, 12, 9, 13, 10, 14, 11]
SCALE_INTRINSIC_PARAMETERS = float(3840./640) # intrinsic parameters have to be scaled to 4K image size

df_keypoints = pd.read_csv(os.path.join(DATA_DIR , "df_kpt"+ ".csv"))
df_pose_local = pd.read_csv(os.path.join(DATA_DIR , "df_pose_local"+ ".csv"))
df_cameras = pd.read_csv(os.path.join(DATA_DIR , "camera_smooth"+ ".csv"))
df_track = pd.read_csv(os.path.join(DATA_DIR , "df_track "+ ".csv"))

In [ ]:
FILE_NAMES = np.unique(df_cameras['file_name'])

In [ ]:
to_homogenous = lambda points_2D: np.hstack((points_2D, np.ones(points_2D.shape[0])[:,None]))

class Transform:
    def __init__(self, Rotation, tvecs):
        self.R = Rotation
        self.t = tvecs
    def inverse(self):
        return Transform(self.R.T, -self.R.T @ self.t)
    def apply(self, points):
        """ Points are assumed to be in format N x d
            where, 'N' are number of points and 'd' being their dimension. 
            Note: Points must be in non-homogenous coordinates
        """
        return points @ self.R.T + self.t
        


class FrameItems:
    """ This class extracts all relevant data per frame.
    """
    def __init__(self, df_keypoints_2D, df_pose_3D_local, df_camera_parameters, track_id, file_name):
        self.df_keypoints_2D = df_keypoints_2D 
        self.df_pose_3D_local = df_pose_3D_local
        self.df_camera_parameters = df_camera_parameters
        self.track_id = track_id
        self.file_name = file_name
        
    def keypoints_2D(self, homogenous=False):
        df_frame = self.df_keypoints_2D
        return np.array(df_frame[["kx","ky"]])
    
    def skin_points_poselocal_3D(self):
        df_frame = self.df_pose_3D_local
        return np.array(df_frame[["kx","ky","kz"]])
    
    def coco_points_poselocal_3D(self):
        df_frame = self.df_pose_3D_local.iloc[SPIN_TO_COCO]
        return np.array(df_frame[["kx","ky","kz"]])
    
    def intrinsics(self):
        df_frame = self.df_camera_parameters
        fx= df_frame.fx.iat[0] * SCALE_INTRINSIC_PARAMETERS
        fy= df_frame.fy.iat[0] * SCALE_INTRINSIC_PARAMETERS
        cx= df_frame.princ_x.iat[0] * SCALE_INTRINSIC_PARAMETERS
        cy= df_frame.princ_y.iat[0] * SCALE_INTRINSIC_PARAMETERS
        return np.array([[fx, 0., cx],
                         [0., fy, cy],
                         [0., 0.,  1.]])

# Extrinsics cam_T_pitch
def extract_extrinsics_from_pandas_df(df_camera_parameters):
    df_frame = df_camera_parameters
    rot_x= df_frame.rot_x.iat[0]
    rot_y= df_frame.rot_y.iat[0]
    rot_z= df_frame.rot_z.iat[0]
        
    t_x= df_frame.tx.iat[0]
    t_y= df_frame.ty.iat[0]
    t_z= df_frame.tz.iat[0]
    R = cv2.Rodrigues(np.array([rot_x, rot_y, rot_z]))[0]
    # multiply with .1 to get in meters from decimeters
    return Transform(R, np.array([  t_x,   t_y,   t_z]) * .1)

# Returns dataset generator
def get_data_for_frame_id(frame_id):
    df_camera_parameters = df_cameras[df_cameras["file_name"] == frame_id]
    if df_camera_parameters.empty:
        # skip frame for which we do not have camera parameters
        return None
    else:
        detection_ids = np.unique(df_keypoints[(df_keypoints.file_name == frame_id)].detection_id)
        frame_extrinsics = extract_extrinsics_from_pandas_df(df_camera_parameters)
        # Tranformation from camera to global pitch space
        pitch_T_cam = frame_extrinsics.inverse()
        for detection_id in detection_ids:
            try:
                df_keypoints_2D = df_keypoints[(df_keypoints.file_name == frame_id) & (df_keypoints.detection_id == detection_id)]
                # corresponding track id associated with detection id could be found using df_track dataframe
                corresponding_track_id = int(df_track[(df_track.file_name == frame_id) & (df_track.detection_id == detection_id)]["track_id"])
                df_pose_3D_local = df_pose_local[(df_pose_local.file_name == frame_id) & (df_pose_local.track_id == corresponding_track_id)]
            
                if df_pose_3D_local.empty or df_keypoints_2D.empty:
                    continue
            except Exception as e:
                # TODO: look into errors.
                continue
        

            frame_items = FrameItems(df_keypoints_2D, df_pose_3D_local, df_camera_parameters, corresponding_track_id, frame_id)
            
            yield detection_id, frame_items, pitch_T_cam

In [ ]:
output_column_names = ['kx','ky','kz','kid','track_id','file_name']
# initialize pandas output data frame
df_outputFile = pd.DataFrame(columns=output_column_names)
def construct_output_df_frame(kx, ky, kz, track_id, file_name):
    df_frame = pd.DataFrame(columns=output_column_names)
    df_frame.kx, df_frame.ky, df_frame.kz  = kx, ky, kz
    df_frame.kid = list(range(len(kx)))
    df_frame.track_id = len(kx) * [track_id]
    df_frame.file_name = len(kx) * file_name
    return df_frame

In [ ]:
# Start time
start = time.time()
for frame_id in FILE_NAMES:

    data_generator = get_data_for_frame_id(frame_id)
    if not data_generator:
        #skip this frame
        continue
    for detection_id, frame_items, pitch_T_cam in data_generator:

        print (f"Processing => detection id: {detection_id}, track_id: {frame_items.track_id} and frame: {frame_id}")
        ''' 
        Use pnp to obtain pose of camera with respect to each object. Knowing atleast 3 corresponce points between 3D-2D one could
        estimate camera pose which has 6 DoF (3 for rotation and 3 for translation). In our case I use COCO point correspondences
        to estimate camera pose. In addition I use RANSAC to remove the effect of outlier for camera pose estimation.
        
        However i do not take into account the error in provided extrinsics.
        '''
        try:
            Success, rotation, tvecs, _ = cv2.solvePnPRansac(frame_items.coco_points_poselocal_3D(), frame_items.keypoints_2D(), frame_items.intrinsics(), distCoeffs=None, flags=cv2.SOLVEPNP_AP3P)
        except Exception as e:
            # TODO: look into errors. For now I skip.
            continue
        if Success:
            cam_T_local = Transform(cv2.Rodrigues(rotation.flatten())[0], tvecs.flatten())
            
            # Use estimated pose of camera to tranform skin points in local to global pitch coordinate space
            skin_points_local_3D =  frame_items.skin_points_poselocal_3D()
            skin_points_cam_3D =  cam_T_local.apply(skin_points_local_3D)
            
            #*****************TODO: bugfix: below peice of code is actually not required. 
            radial_distances = np.linalg.norm(skin_points_cam_3D, axis =1)
            back_projected_pixel_2d = np.squeeze(cv2.projectPoints(skin_points_local_3D, rotation, tvecs, frame_items.intrinsics(), distCoeffs=None)[0])
            K_inverse = np.linalg.inv(frame_items.intrinsics())
            normalized_cam = normalize(to_homogenous(back_projected_pixel_2d) @ K_inverse.T)
            skin_points_cam_3D_ = normalized_cam * radial_distances[:, None]
            #**********************************************************************
            
            skin_points_pitch_3D = pitch_T_cam.apply(skin_points_cam_3D_)
            
            # scale to decimeters before saving
            df_frame= construct_output_df_frame(skin_points_pitch_3D[:,0] * 10, 
                                                skin_points_pitch_3D[:,1] * 10, 
                                                skin_points_pitch_3D[:,2] * 10,
                                                frame_items.track_id, frame_items.file_name)
            #print(df_frame)
            #print(skin_points_cam_3D)
            df_outputFile = df_outputFile.append(df_frame)

      
end = time.time()
seconds = end - start
# Time elapsed
print (f"Time taken : {seconds} seconds")
# Calculate frames per second
fps  = (frame_id +1) / seconds
print(f"Estimated frames per second approximately: {fps}")

        


        

    
            

Processing => detection id: 0, track_id: 0 and frame: 0
Processing => detection id: 3, track_id: 3 and frame: 0
Processing => detection id: 4, track_id: 4 and frame: 0
Processing => detection id: 6, track_id: 6 and frame: 0
Processing => detection id: 7, track_id: 7 and frame: 0
Processing => detection id: 8, track_id: 8 and frame: 0
Processing => detection id: 9, track_id: 9 and frame: 0
Processing => detection id: 10, track_id: 10 and frame: 0
Processing => detection id: 11, track_id: 11 and frame: 0
Processing => detection id: 12, track_id: 12 and frame: 0
Processing => detection id: 13, track_id: 13 and frame: 0
Processing => detection id: 14, track_id: 14 and frame: 0
Processing => detection id: 15, track_id: 15 and frame: 0
Processing => detection id: 16, track_id: 16 and frame: 0
Processing => detection id: 17, track_id: 17 and frame: 0
Processing => detection id: 18, track_id: 18 and frame: 0
Processing => detection id: 0, track_id: 0 and frame: 1
Processing => detection id: 2,

Processing => detection id: 0, track_id: 15 and frame: 9
Processing => detection id: 1, track_id: 6 and frame: 9
Processing => detection id: 2, track_id: 0 and frame: 9
Processing => detection id: 3, track_id: 9 and frame: 9
Processing => detection id: 5, track_id: 7 and frame: 9
Processing => detection id: 6, track_id: 3 and frame: 9
Processing => detection id: 7, track_id: 8 and frame: 9
Processing => detection id: 9, track_id: 10 and frame: 9
Processing => detection id: 10, track_id: 4 and frame: 9
Processing => detection id: 12, track_id: 12 and frame: 9
Processing => detection id: 13, track_id: 11 and frame: 9
Processing => detection id: 14, track_id: 13 and frame: 9
Processing => detection id: 15, track_id: 14 and frame: 9
Processing => detection id: 16, track_id: 18 and frame: 9
Processing => detection id: 17, track_id: 17 and frame: 9
Processing => detection id: 18, track_id: 16 and frame: 9
Processing => detection id: 0, track_id: 6 and frame: 10
Processing => detection id: 1,

Processing => detection id: 8, track_id: 8 and frame: 17
Processing => detection id: 9, track_id: 11 and frame: 17
Processing => detection id: 10, track_id: 10 and frame: 17
Processing => detection id: 12, track_id: 13 and frame: 17
Processing => detection id: 13, track_id: 18 and frame: 17
Processing => detection id: 15, track_id: 14 and frame: 17
Processing => detection id: 16, track_id: 17 and frame: 17
Processing => detection id: 17, track_id: 15 and frame: 17
Processing => detection id: 18, track_id: 16 and frame: 17
Processing => detection id: 0, track_id: 0 and frame: 18
Processing => detection id: 1, track_id: 6 and frame: 18
Processing => detection id: 2, track_id: 7 and frame: 18
Processing => detection id: 3, track_id: 19 and frame: 18
Processing => detection id: 4, track_id: 15 and frame: 18
Processing => detection id: 6, track_id: 3 and frame: 18
Processing => detection id: 7, track_id: 8 and frame: 18
Processing => detection id: 8, track_id: 4 and frame: 18
Processing => 

Processing => detection id: 7, track_id: 11 and frame: 25
Processing => detection id: 8, track_id: 19 and frame: 25
Processing => detection id: 9, track_id: 13 and frame: 25
Processing => detection id: 10, track_id: 8 and frame: 25
Processing => detection id: 11, track_id: 4 and frame: 25
Processing => detection id: 12, track_id: 14 and frame: 25
Processing => detection id: 14, track_id: 17 and frame: 25
Processing => detection id: 15, track_id: 6 and frame: 25
Processing => detection id: 16, track_id: 16 and frame: 25
Processing => detection id: 18, track_id: 10 and frame: 25
Processing => detection id: 19, track_id: 12 and frame: 25
Processing => detection id: 0, track_id: 15 and frame: 26
Processing => detection id: 1, track_id: 9 and frame: 26
Processing => detection id: 2, track_id: 0 and frame: 26
Processing => detection id: 3, track_id: 7 and frame: 26
Processing => detection id: 4, track_id: 20 and frame: 26
Processing => detection id: 5, track_id: 11 and frame: 26
Processing =

Processing => detection id: 18, track_id: 16 and frame: 32
Processing => detection id: 19, track_id: 13 and frame: 32
Processing => detection id: 0, track_id: 0 and frame: 33
Processing => detection id: 1, track_id: 6 and frame: 33
Processing => detection id: 2, track_id: 3 and frame: 33
Processing => detection id: 3, track_id: 24 and frame: 33
Processing => detection id: 4, track_id: 18 and frame: 33
Processing => detection id: 5, track_id: 7 and frame: 33
Processing => detection id: 7, track_id: 14 and frame: 33
Processing => detection id: 8, track_id: 20 and frame: 33
Processing => detection id: 9, track_id: 11 and frame: 33
Processing => detection id: 10, track_id: 13 and frame: 33
Processing => detection id: 11, track_id: 10 and frame: 33
Processing => detection id: 12, track_id: 4 and frame: 33
Processing => detection id: 13, track_id: 9 and frame: 33
Processing => detection id: 14, track_id: 8 and frame: 33
Processing => detection id: 15, track_id: 17 and frame: 33
Processing =>

...

Processing => detection id: 4, track_id: 3 and frame: 90
Processing => detection id: 5, track_id: 12 and frame: 90
Processing => detection id: 6, track_id: 8 and frame: 90
Processing => detection id: 7, track_id: 9 and frame: 90
Processing => detection id: 8, track_id: 28 and frame: 90
Processing => detection id: 9, track_id: 15 and frame: 90
Processing => detection id: 10, track_id: 0 and frame: 90
Processing => detection id: 11, track_id: 29 and frame: 90
Processing => detection id: 13, track_id: 7 and frame: 90
Processing => detection id: 14, track_id: 17 and frame: 90
Processing => detection id: 15, track_id: 21 and frame: 90
Processing => detection id: 16, track_id: 4 and frame: 90
Processing => detection id: 17, track_id: 13 and frame: 90
Processing => detection id: 18, track_id: 11 and frame: 90
Processing => detection id: 19, track_id: 10 and frame: 90
Processing => detection id: 20, track_id: 16 and frame: 90
Processing => detection id: 0, track_id: 24 and frame: 91
Processing

Processing => detection id: 14, track_id: 21 and frame: 97
Processing => detection id: 15, track_id: 16 and frame: 97
Processing => detection id: 16, track_id: 15 and frame: 97
Processing => detection id: 17, track_id: 7 and frame: 97
Processing => detection id: 19, track_id: 11 and frame: 97
Processing => detection id: 21, track_id: 17 and frame: 97
Processing => detection id: 0, track_id: 9 and frame: 98
Processing => detection id: 1, track_id: 12 and frame: 98
Processing => detection id: 3, track_id: 24 and frame: 98
Processing => detection id: 4, track_id: 4 and frame: 98
Processing => detection id: 5, track_id: 0 and frame: 98
Processing => detection id: 6, track_id: 13 and frame: 98
Processing => detection id: 7, track_id: 14 and frame: 98
Processing => detection id: 8, track_id: 3 and frame: 98
Processing => detection id: 10, track_id: 25 and frame: 98
Processing => detection id: 11, track_id: 8 and frame: 98
Processing => detection id: 12, track_id: 29 and frame: 98
Processing 

Processing => detection id: 12, track_id: 25 and frame: 105
Processing => detection id: 13, track_id: 28 and frame: 105
Processing => detection id: 14, track_id: 3 and frame: 105
Processing => detection id: 15, track_id: 14 and frame: 105
Processing => detection id: 16, track_id: 21 and frame: 105
Processing => detection id: 18, track_id: 16 and frame: 105
Processing => detection id: 0, track_id: 12 and frame: 106
Processing => detection id: 1, track_id: 24 and frame: 106
Processing => detection id: 2, track_id: 0 and frame: 106
Processing => detection id: 3, track_id: 15 and frame: 106
Processing => detection id: 5, track_id: 29 and frame: 106
Processing => detection id: 6, track_id: 17 and frame: 106
Processing => detection id: 7, track_id: 28 and frame: 106
Processing => detection id: 8, track_id: 9 and frame: 106
Processing => detection id: 9, track_id: 25 and frame: 106
Processing => detection id: 11, track_id: 8 and frame: 106
Processing => detection id: 12, track_id: 3 and frame

Processing => detection id: 11, track_id: 4 and frame: 113
Processing => detection id: 12, track_id: 14 and frame: 113
Processing => detection id: 13, track_id: 25 and frame: 113
Processing => detection id: 15, track_id: 21 and frame: 113
Processing => detection id: 16, track_id: 11 and frame: 113
Processing => detection id: 17, track_id: 37 and frame: 113
Processing => detection id: 19, track_id: 16 and frame: 113
Processing => detection id: 0, track_id: 15 and frame: 114
Processing => detection id: 1, track_id: 24 and frame: 114
Processing => detection id: 3, track_id: 13 and frame: 114
Processing => detection id: 4, track_id: 17 and frame: 114
Processing => detection id: 5, track_id: 36 and frame: 114
Processing => detection id: 6, track_id: 9 and frame: 114
Processing => detection id: 7, track_id: 12 and frame: 114
Processing => detection id: 8, track_id: 8 and frame: 114
Processing => detection id: 9, track_id: 28 and frame: 114
Processing => detection id: 10, track_id: 14 and fra

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\calib3d\src\solvepnp.cpp:216: error: (-215:Assertion failed) npoints >= 4 && npoints == std::max(ipoints.checkVector(2, CV_32F), ipoints.checkVector(2, CV_64F)) in function 'cv::solvePnPRansac'


In [ ]:
df_outputFile.to_csv(os.path.join(DATA_DIR, "df_pose_global"+ ".csv"), encoding='utf-8', index=False)

In [ ]:
df_outputFile

,kx,ky,kz,kid,track_id,file_name
0,759.369689,401.179700,-19.065866,0,0,0
1,760.956548,400.427866,-17.823250,1,0,0
2,760.292358,398.704007,-16.585064,2,0,0
3,760.453234,399.027843,-13.816598,3,0,0
4,759.368806,400.072176,-11.381529,4,0,0
...,...,...,...,...,...,...
44,560.280498,583.908986,-13.017313,44,17,5782
45,559.942694,583.834866,-13.558874,45,17,5782
46,560.465995,584.244524,-13.545624,46,17,5782
47,559.106406,584.334002,-13.815007,47,17,5782


In [ ]:
frame_items.coco_points_poselocal_3D().shape[0]>4

True